# Tools for Analytics Final Project ###

Jaeseop Shin / js6364  &. Hyunjin Jun / hj2642

## Environment Setup

In [4]:
# Importing all libraries used for the project

import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import geopandas as gpd
import requests
import psycopg2



from bs4 import BeautifulSoup
from sodapy import Socrata
from sqlalchemy import create_engine

# SQL Data
db_username = ""

## Part 1. Data Processing

### Data Setup

In [7]:
app_token = "2Hn2wwabCLXVYhGN4b9tEtJ11"

# Downloading NYC 311 data
url_311 = "data.cityofnewyork.us"
set_311 = "erm2-nwe9"
client_311 = Socrata(url_311, app_token)
client_311.timeout = 60
# Filter
where_311 = "filter_conditions"
select_311 = "filter_conditions"
# Data to CSV
data_311 = client_311.get(set_311, where=where_311, limit=2000)
df_311 = pd.DataFrame.from_records(data_311)
df_311.to_csv("nyc_311.csv")

# Downloading NYC Tree data
url_tree = "data.cityofnewyork.us"
set_tree = "uvpi-gqnh"
client_tree = Socrata(url_tree, app_token)
client_tree.timeout = 60
# Filter
where_tree = "filter_conditions"
select_tree = "filter_conditions"
# Data to CSV
data_tree = client_tree.get(set_tree, where=where_tree, limit=2000)
df_tree = pd.DataFrame.from_records(data_tree)
df_tree.to_csv("nyc_tree.csv")


rent_path = "/Users/ariez/Columbia/Tools for Analytics/GIT/TfA_FinalProject/data/zillow_rent_data.csv"          # Should be altered by users
nyc_zipcodes_shp = "/Users/ariez/Columbia/Tools for Analytics/GIT/TfA_FinalProject/data/nyc_zipcodes.shp"       # Should be altered by users

# Coordiante Reference System
CRS = 4326

### Data Cleaning & Filtering 